In [20]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import requests

class GPT2Layer(nn.Module):
    def __init__(self, embed_size, heads, ff_hidden_dim):
        super(GPT2Layer, self).__init__()
        self.attention = MultiHeadAttention(embed_size, heads)
        self.feedforward = FeedForward(embed_size, ff_hidden_dim)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

    def forward(self, value, key, query, mask):
        attention_output = self.attention(value, key, query, mask)
        x = self.norm1(attention_output + query)
        forward_output = self.feedforward(x)
        out = self.norm2(forward_output + x)
        return out

class GPT2Model(nn.Module):
    def __init__(self, vocab_size, embed_size, max_len, num_layers, heads, ff_hidden_dim, device):
        super(GPT2Model, self).__init__()
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.device = device

        self.token_embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_len, embed_size)
        self.layers = nn.ModuleList([
            GPT2Layer(embed_size, heads, ff_hidden_dim) for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(embed_size, vocab_size)

    def forward(self, x, mask=None):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

        out = self.token_embedding(x) + self.positional_embedding(positions)

        if mask is not None:
            mask = mask.to(self.device)

        for layer in self.layers:
            out = layer(out, out, out, mask)

        out = self.fc_out(out)
        return out






class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        
    def forward(self, values, keys, query, mask):
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.nn.functional.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, embed_size, ff_hidden_dim):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(embed_size, ff_hidden_dim)
        self.fc2 = nn.Linear(ff_hidden_dim, embed_size)

    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class TextDataset(Dataset):
    def __init__(self, data, vocab, max_len=512):
        self.data = data
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]
        tokens = [self.vocab[char] for char in text][:self.max_len]
        input_ids = torch.tensor(tokens, dtype=torch.long)
        return input_ids


# Download the data
data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
response = requests.get(data_url)
text_data = response.text

# Create vocab
vocab = {char: idx for idx, char in enumerate(set(text_data))}
vocab_size = len(vocab)

# Create datasets and data loaders
dataset = TextDataset([text_data], vocab)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Model and optimizer initialization
embed_size = 256
heads = 8
num_layers = 4
ff_hidden_dim = 512
max_len = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT2Model(vocab_size, embed_size, max_len, num_layers, heads, ff_hidden_dim, device).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)


# Training loop
# Training loop
# ...

# Training loop
criterion = nn.CrossEntropyLoss()
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for input_ids in data_loader:
        input_ids = input_ids.to(device)

        # Use input_ids.size(0) instead of input_ids.size(1) for mask creation
        mask = torch.triu(torch.ones(input_ids.size(0), input_ids.size(0))).transpose(0, 1).type(dtype=torch.bool)
        mask = mask.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids)
        # Note: No need to specify mask here, as GPT2 inherently attends to all positions
        loss = criterion(outputs.view(-1, vocab_size), input_ids.view(-1))
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print(model)

Epoch [1/5], Loss: 4.2599
Epoch [2/5], Loss: 4.0605
Epoch [3/5], Loss: 3.8662
Epoch [4/5], Loss: 3.6782
Epoch [5/5], Loss: 3.4979
GPT2Model(
  (token_embedding): Embedding(65, 256)
  (positional_embedding): Embedding(512, 256)
  (layers): ModuleList(
    (0-3): 4 x GPT2Layer(
      (attention): MultiHeadAttention(
        (values): Linear(in_features=32, out_features=32, bias=False)
        (keys): Linear(in_features=32, out_features=32, bias=False)
        (queries): Linear(in_features=32, out_features=32, bias=False)
        (fc_out): Linear(in_features=256, out_features=256, bias=True)
      )
      (feedforward): FeedForward(
        (fc1): Linear(in_features=256, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (fc_out): Linear(in_features=256, out_features=65, bias=True)
)


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained model and tokenizer
model_name = 'gpt2'  # You can change this to 'gpt2-medium', 'gpt2-large', etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


In [4]:
# Encode input text
input_text = "madhu is a good boy"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate predictions
with torch.no_grad():
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


/opt/conda/miniconda3/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


madhu is a good boy. He is very good at his job. I am not sure if he is going to be able to do it.

"I am sure he will be fine. But I don't know if it will work
